In [11]:
import urllib.request as urllib
import zipfile
import pandas as pd
pd.set_option('display.max_columns', None)
import os
import matplotlib.pyplot as plt
import time
import numpy as np
states = ["AL", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", "MA", "MI", "MN", "MS", 
          "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", 
          "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [3]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [4]:
url = 'https://gaftp.epa.gov/air/nei/2017/data_summaries/2017v1/2017neiJan_county_tribe_allsector.zip'
filehandle, _ = urllib.urlretrieve(url)
zip_file_object = zipfile.ZipFile(filehandle, 'r')
my_file = zip_file_object.namelist()[1]
df = pd.read_csv(zip_file_object.open(my_file))
zip_file_object.close()

In [7]:
my_file = zip_file_object.namelist()[1]
df = pd.read_csv(zip_file_object.open(my_file))
zip_file_object.close()

In [9]:
df = df.filter(['state', 'fips code', 'county', 'sector', 'pollutant desc','pollutant type(s)', 'total emissions', 'emissions uom'])

In [12]:
df['pollutant type(s)'].unique()

array(['HAP', 'CAP', nan, 'CAP/HAP', 'GHG'], dtype=object)

In [14]:
dfghg = df[df['pollutant type(s)']=='GHG']
dfcap = df[df['pollutant type(s)']=='CAP']

In [22]:
dfcap.head()

,state,fips code,county,sector,pollutant desc,pollutant type(s),total emissions,emissions uom
8,GA,13201,Miller,Mobile - On-Road Diesel Light Duty Vehicles,Sulfur Dioxide,CAP,0.016603,TON
11,GA,13205,Mitchell,Gas Stations,Volatile Organic Compounds,CAP,83.047791,TON
20,TN,47001,Anderson,Fuel Comb - Comm/Institutional - Oil,PM Condensible,CAP,0.001739,TON
21,NC,37055,Dare,Mobile - On-Road Diesel Heavy Duty Vehicles,Sulfur Dioxide,CAP,0.362939,TON
26,GA,13071,Colquitt,Mobile - On-Road Diesel Heavy Duty Vehicles,PM10 Primary (Filt + Cond),CAP,17.391094,TON


In [ ]:
cap_sums = pd.DataFrame(columns=list(dfcap.columns))